***UFLP***
- *I* korisnika, *J* resursa
- $c_{ij}$ - cena dodeljivanja resursa *j* korisniku *i*
- $f_j$ - cena uspostavljanja resursa *j* \\
- Resenje: koje bolnice treba da se sagrade \\
- Kodiranje: niz nula i jedinica (tj. True ili False)
- Fitness: hocemo sto manju sumu $c_{ij}$ i $f_j$

In [1]:
import random

In [3]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for i in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]

        return numUsers, numResources, cost, fixedCost

In [5]:
numUsers, numResources, cost, fixedCost = readInput('in.txt')

In [9]:
class Individual():
    def __init__(self, numUsers, numResources, cost, fixedCost):
        # kodiranje
        self.code = [random.random() < 0.25 for _ in range(numResources)]
        # provera da li je resenje (ne)dopustivo
        self.correctNonFeasible()
        # fitnes
        self.fitness = self.calculateFitness(cost, fixedCost)

    def __lt__(self, other):
        return self.fitness < other.fitness

    def correctNonFeasible(self):
        for c in self.code:
            if c: # nasli smo jedan True
                return 
 
        # nedopustivo resenje - random indeks stavljamo na True
        index = random.randrange(0, len(self.code))
        self.code[index] = True

    def calculateFitness(self, cost, fixedCost):
        numUsers = len(cost)
        numResources = len(fixedCost)

        value = 0.0
        used = [False for _ in range(numResources)]

        # dodajemo cenu dodeljivanja
        for i in range(numUsers):
            minCost = float('inf')
            usedResource = -1

            for j in range(numResources):
                if self.code[j] and cost[i][j] < minCost:
                    minCost = cost[i][j]
                    usedResource = j

            value += minCost
            used[usedResource] = True

        # dodajemo cenu uspostavljanja
        for i in range(numResources):
            if used[i]:
                value += fixedCost[i]

        self.code = used
        
        return value

In [13]:
POPULATION_SIZE = 100
ELITISM_SIZE = int(0.3*POPULATION_SIZE) # 30%
MAX_ITER = 500
TOURNAMENT_SIZE = 6
MUTATION_PROB = 0.05 # 5%

In [11]:
def selection(population):
    minFitness = float('inf')
    bestIndex = -1

    for i in range(TOURNAMENT_SIZE):
        index = random.randrange(len(population))

        if population[index].fitness < minFitness:
            bestIndex = index

    return bestIndex

In [12]:
def crossover(parent1, parent2, child1, child2):
    breakpoint = random.randrange(0, len(parent1.code))

    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child2.code[:breakpoint] = parent2.code[:breakpoint]

    child1.code[breakpoint:] = parent2.code[breakpoint:]
    child2.code[breakpoint:] = parent1.code[breakpoint:]

    # ako su nedopustiva resenja
    child1.correctNonFeasible()
    child2.correctNonFeasible()

In [17]:
def mutation(child):
    for i in range(len(child.code)):
        if random.random() < MUTATION_PROB:
            child.code[i] = not child.code[i]

    # ako je resenje nedopustivo
    child.correctNonFeasible()

In [18]:
population = [Individual(numUsers, numResources, cost, fixedCost) for i in range(POPULATION_SIZE)]
newPopulation = [Individual(numUsers, numResources, cost, fixedCost) for i in range(POPULATION_SIZE)]

for i in range(MAX_ITER):
    population.sort() # poziva __lt__

    # cuvamo ELITISM_SIZE najboljih jedinki
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]

    for j in range(ELITISM_SIZE, POPULATION_SIZE, 2):
        parent1Index = selection(population)
        parent2Index = selection(population)

        # ukrstanje roditelja
        crossover(population[parent1Index], population[parent2Index], newPopulation[j], newPopulation[j+1])

        # mutacija dece
        mutation(newPopulation[j])
        mutation(newPopulation[j+1])

        # fitnes dece
        newPopulation[j].fitness = newPopulation[j].calculateFitness(cost, fixedCost)
        newPopulation[j+1].fitness = newPopulation[j+1].calculateFitness(cost, fixedCost)

    population = newPopulation

bestIndividual = min(population) # poziva __lt__
print('Solution: {}, fitness: {}'.format(bestIndividual.code, bestIndividual.fitness))

Solution: [True, False, False], fitness: 34.0
